In [ ]:
output_dir = '/tmp/citations/'

In [ ]:
import json, requests, urllib, os
from datetime import datetime, date

In [ ]:
doi = "https://openalex.org/W2000785269"

In [ ]:
def get_json(doi):
    ## Construct API Call
    #doi = input("Enter a DOI, OpenAlex, MAG, PMID, or PMCID: ")
    apicall = ('https://api.openalex.org/works/{}'.format(doi))
    #  Open api call url and assign to variable
    dirtywork = urllib.request.urlopen(apicall)
    #  Convert to json
    work = json.load(dirtywork)
    return work

In [ ]:
work = get_json(doi)

In [ ]:
fname = (work['id'].replace("https://openalex.org/", ""))

In [ ]:
def get_citations(work):
    
        #fname = work['id'].replace("https://openalex.org/","")
    reference_list = []
    for ref in work['referenced_works']:
        reference_list.append(ref)
    return reference_list


In [ ]:
referenced_works = get_citations(work)

In [ ]:
print(referenced_works)

In [ ]:
def bibtext_oa_conversion(oaid):

    ## call the OpenAlex api for a JSON of the work metadata
    apicall = ('https://api.openalex.org/works/{}'.format(oaid))
    f = urllib.request.urlopen(apicall)
    oa_work = json.load(f)
    oa_work_pub_date = date.fromisoformat(oa_work['publication_date'])

    ### MAPPING DATA FROM OPENALEX TO BIBTEX FIELDS ###

    ## "type" field value error handling
    def is_in_bibtex_entry(my_key):
        for key, val in bibtex_entry_type_map.items():
            if my_key == None:
                return 'null'
            elif my_key in key:
                return val
    ## set up "author" field value
    def get_author(author_field_position):
        author_field = ''
        author_list = []
        if oa_work['authorships']:
            if author_field_position == 'citekey':
                author_field = str(oa_work['authorships'][0]['author']['display_name'])
                return author_field
            else:
                for i in oa_work['authorships']:
                    author_list.append(i['author']['display_name'])
                for j in range(0, len(author_list) - 1):
                    author_field += str(author_list[j]) + " and "
                author_field += author_list[-1]
                return author_field
        else:
            author_field == 'null'
            return author_field

    citekey_author = 'citekey'
    citation_authors = 'citation'

    ## BibTeX entry "type" mapping. OpenAlex uses the Crossref controlled vocabulary for works "types", but this field is
    ## under development by the OpenAlex team. This is my best approximation for a crosswalk between
    ## BibTeX and OpenAlex (Crossref) works "types".
    bibtex_entry_type_map = {'journal-article': 'article',
                             ('book-section', 'monograph'): 'book',
                             ('book-track', 'book-part', 'book-chapter', 'book-series'): 'inbook',
                             ('proceedings-article', 'proceedings-series'): 'inproceedings',
                             'dissertation': 'phdthesis'}


    ## check for the presence of page numbers and set up the "page numbers" fields
    oa_first_page = oa_work['biblio']['first_page']
    oa_last_page = oa_work['biblio']['last_page']
    oa_page_numbers_draft = str(oa_first_page) + ', ' + str(oa_last_page)
    oa_page_numbers = ''

    if oa_page_numbers_draft == 'None, None':
        oa_page_numbers = 'null'
    else:
        oa_page_numbers = oa_page_numbers_draft

    ## BibTeX Dictionary of mapped values
    bibtex_mapping = {'title': oa_work['title'],
                      'author': get_author(citation_authors),
                      'publisher': oa_work['host_venue']['publisher'],
                      'doi': oa_work['doi'],
                      'issn': oa_work['host_venue']['issn_l'],
                      'journal': oa_work['host_venue']['display_name'],
                      'month': oa_work_pub_date.strftime("%b").lower(),
                      'number': oa_work['biblio']['issue'],
                      'pages': oa_page_numbers,
                      'type': is_in_bibtex_entry(oa_work['type']),
                      'url': oa_work['host_venue']['url'],
                      'volume': oa_work['biblio']['volume'],
                      'year': oa_work['publication_year'],
                      'note' : oa_work['id']}


    ### CONSTRUCTING THE BIBTEX CITATION ###

    ## set up the citation key and pretext

    if get_author(citekey_author):
        bibtex_citekey = get_author(citekey_author).split()[-1] + \
                         str(oa_work['publication_year']) + \
                         str((oa_work['title']).split()[0])
    else:
        bibtex_citekey = str(oa_work['title'].split()[0]) + \
                     str(oa_work['publication_year']) + \
                     str((oa_work['title']).split()[-1])

    bibtex_entrytype = (oa_work['type'])
    bibtex_pretext = '@' + str(is_in_bibtex_entry(bibtex_entrytype)) + '{' + bibtex_citekey + ",\n"
    bibtex_citation_str = bibtex_pretext

    ## error handling for missing data in the OpenAlex JSON and
    ## constructing the BibTeX citation string
    for i, j in bibtex_mapping.items():
        if i == None:
            continue
        elif j == None or j == 'null':
            continue
        else:
            line = str(i) + " = " + '{' + str(j) + "},\n"
            bibtex_citation_str += line
    bibtex_citation_str += '}'

    ## return the completed BibTeX citation as a string
    return bibtex_citation_str

now = datetime.now()
now = now.strftime('%d%m%y%H%M%S')
#print(now)
#file = open(output_dir + fname + ".bib", 'a', encoding='utf-8')

In [ ]:
oarefs = get_citations(work)
for i in oarefs:
    print(i)

In [ ]:
## print and write to file all BibTeX citations
with open (os.path.join(output_dir + "test.bib"), 'w', encoding='utf-8') as my_file:
    print(my_file)
    print(output_dir)
    for i in oarefs:
        #bibtex_citation = bibtext_oa_conversion(i)
        #print(bibtex_citation + '\n')
        #my_file.writelines(bibtex_citation + "\n\n")
        my_file.write(i + "\n\n")